El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [1]:
import pandas
from IPython.display import display

En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [16]:
data_Konex = pandas.read_csv('./DataValidada_PilotoKonex.csv')

# Creamos un calendario anidado con la siguiente estructura: Indice + Nombre Seccion + [preguntas] ;
# Indice + TituloPregunta + [opciones] + value ; Indice + Opcion

interprete = {}

seccion_formacion = {'Indice':1,'Nombre':'Formacion','Preguntas':[]}
pregunta_formacion = {'Indice':1,'Pregunta':'Formacion','Opciones':[]}

seccion_formacion['Preguntas'].append(pregunta_formacion)
interprete[seccion_formacion['Indice']]=seccion_formacion


In [18]:
interprete

{1: {'Indice': 1,
  'Nombre': 'Formacion',
  'Preguntas': [{'Indice': 1, 'Pregunta': 'Formacion', 'Opciones': []}]}}

In [35]:
import re

secciones = 0
preguntas = 0
opciones = 0
nivelActual = 0
interprete = {}

with open('Interprete.txt') as f:
    lineas = f.readlines()
for linea in lineas:
    tabs = len(re.findall("\t", linea))
    if tabs < nivelActual:
        if tabs == 1:
            opciones = 0
        if tabs == 0:
            preguntas = 0
            opciones = 0
    if tabs == 0:
        seccion = {}
        secciones = secciones + 1
        seccion['Indice'] = secciones
        seccion['Preguntas'] = {}
        seccion['Titulo'] = linea[:-1]
        interprete[secciones] = seccion
        
        

In [36]:
interprete

{1: {'Indice': 1, 'Preguntas': {}, 'Titulo': 'Formacion'},
 2: {'Indice': 2, 'Preguntas': {}, 'Titulo': 'Ver'},
 3: {'Indice': 3, 'Preguntas': {}, 'Titulo': 'Justificacion'}}

In [2]:


dict_KonexFormacion = {'1_1_0':'Artes',
                  '1_1_1':'Biomedicas',
                  '1_1_2':'Economicas',
                  '1_1_3':'Ciencias Exactas',
                  '1_1_4':'Fisica',
                  '1_1_5':'Ciencias Sociales',
                  '1_1_6':'Derecho',
                  '1_1_7':'Otra carrera',
                  '1_1_8':'Psicologia',
                  '1_1_9':'Ninguna',
                  '1_1_10':'Otra(campolibre)'}
dict_KonexExplicaciones = {'2_1_0':'Calor',
                  '2_1_1':'Telequinesis',
                  '2_1_2':'Magnetismo',
                  '2_1_3':'Energia Cosmica',
                  '2_1_4':'Presion',
                  '2_1_5':'Fisica Cuantica',
                  '2_1_6':'Electricidad',
                  '2_1_7':'No lo se'}
dict_KonexConfianza = {'3_1':'Confianza'}
dict_KonexTruco =  {'3_2_0':'Si',
                  '3_2_1':'No',
                  '3_2_2':'NoSe'}
dict_Konex = {}
dict_Konex.update(dict_KonexFormacion)
dict_Konex.update(dict_KonexExplicaciones)
dict_Konex.update(dict_KonexConfianza)
dict_Konex.update(dict_KonexTruco)

data_Konex.rename(columns=dict_Konex,inplace=True)

#data_Konex = data_Konex[data_Konex.sum(axis=1) - data_Konex["Confianza"] == 3] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
data_Konex = data_Konex[data_Konex[list(dict_KonexFormacion.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
data_Konex = data_Konex[data_Konex[list(dict_KonexExplicaciones.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
data_Konex = data_Konex[data_Konex[list(dict_KonexTruco.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta

pepe = data_Konex.stack()
pepe = pepe[(pepe==1) | (pepe.index.get_level_values(1)=="Confianza")]
pepe = pepe.reset_index()
pepe = pepe.rename(columns={'level_0':'IdSujeto','level_1':'valorElegido'})

pepe["NombreColumna"] = pepe.index.map(lambda x: x % 4)
pepe['NombreColumna'].replace([0,1,2,3],['Formacion','Mecanismo','Confianza','Truco'], inplace=True)
condicion = pepe['NombreColumna']=='Confianza'
pepe.loc[condicion,['valorElegido']] = pepe.loc[condicion,[0]].values 

TablaFinal = pepe.pivot(index="IdSujeto", columns="NombreColumna", values="valorElegido")



In [79]:
display (TablaFinal.head())
#display(TablaFinal.pivot_table(rows='Mecanismo', cols='Formacion', values='Truco', aggfunc=len).fillna(0))
#display(TablaFinal.head())


NombreColumna,Confianza,Formacion,Mecanismo,Truco
IdSujeto,,,,
0,4,Biomedicas,Telequinesis,Si
1,10,Ciencias Sociales,No lo se,NoSe
3,7,Artes,Magnetismo,No
4,8,Artes,Presion,NoSe
5,7,Ciencias Sociales,Magnetismo,Si


In [25]:
#g = sns.FacetGrid(TablaFinal, col='Formacion', row='Truco', xlim=[0, 8])
#g.map_dataframe(sns.countplot, y='Mecanismo', order=dict_KonexExplicaciones.values(), palette="dark")
#g.map(sns.countplot, x='Mecanismo')
sns.countplot(data=TablaFinal, x='Mecanismo', hue='Confianza')
plt.show()
sns.barplot(x="Mecanismo", y="Confianza", data=TablaFinal)
plt.show()

In [22]:
sns.set_style("whitegrid")
sns.barplot(x="Mecanismo", y="Confianza", data=TablaFinal)
plt.show()

In [17]:
sns.set_style("whitegrid")
tips = sns.load_dataset("tips")
display (tips)
ax = sns.barplot(x="day", y="total_bill", data=tips)
plt.show()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2
